***
**Exercice**
***

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df_pop_communes = pd.read_csv('https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson6/population_communes.csv', sep=',')
df_surf_departments = pd.read_csv('https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson6/surface_departements.csv', sep=',')

In [3]:
df_pop_communes.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [4]:
df_surf_departments.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [24]:
mask = df_pop_communes['variable'] == 'Population municipale'
df_final_1 = df_pop_communes[mask] \
                           .groupby(['Code département'])['value'] \
                           .sum().astype(float) \
                           .reset_index()
df_final_1 = df_final_1.rename({'Code département': 'code_department',
                                'value': 'sum_pop_municipal'}, axis=1)
df_final_1.head()

,code_department,sum_pop_municipal
0,01,638425.0
1,02,536136.0
2,03,339384.0
3,04,162565.0
4,05,141107.0


In [25]:
mask = df_pop_communes['variable'] == 'Population totale'
df_final_2 = df_pop_communes[mask] \
                          .groupby(['Code département'])['value'] \
                          .sum().astype(float) \
                          .reset_index()
df_final_2 = df_final_2.rename({'Code département': 'code_department',
                            'value': 'sum_pop_total'}, axis=1)
df_final_2.head()

,code_department,sum_pop_total
0,01,655171.0
1,02,549587.0
2,03,349336.0
3,04,167331.0
4,05,146148.0


In [26]:
pd.testing.assert_index_equal(df_final_1.index, df_final_1.index) == None

True

In [30]:
df_final = pd.merge(left=df_final_1, right=df_final_2, on='code_department')
df_final.head()

,code_department,sum_pop_municipal,sum_pop_total
0,01,638425.0,655171.0
1,02,536136.0,549587.0
2,03,339384.0,349336.0
3,04,162565.0,167331.0
4,05,141107.0,146148.0


In [33]:
df_final['pop_ratio_mun_total'] = df_final['sum_pop_municipal'] / df_final['sum_pop_total']

In [37]:
df_pop_communes.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [50]:
df_final.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total
0,01,638425.0,655171.0,0.974440
1,02,536136.0,549587.0,0.975525
2,03,339384.0,349336.0,0.971512
3,04,162565.0,167331.0,0.971518
4,05,141107.0,146148.0,0.965508


In [64]:
df_final_bis = df_final.merge(df_pop_communes[['Code département', 'Code région']].drop_duplicates(),
               left_on='code_department', right_on='Code département',
                             how='left') \
                       .drop(['Code département'], axis=1)
df_final_bis = df_final_bis.rename({'Code région': 'code_region'}, axis=1)

In [65]:
df_final_bis.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region
0,01,638425.0,655171.0,0.974440,84
1,02,536136.0,549587.0,0.975525,32
2,03,339384.0,349336.0,0.971512,84
3,04,162565.0,167331.0,0.971518,93
4,05,141107.0,146148.0,0.965508,93


In [66]:
mask = df_pop_communes['variable'] == 'Population municipale'
codeRegionPop = df_pop_communes[mask].groupby('Code région').value.sum().reset_index()
codeRegionPop.columns = ['code_region', 'region_pop_municipal']

In [70]:
df_final_alt = df_final_bis.merge(codeRegionPop, on='code_region')
df_final_alt.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,region_pop_municipal
0,01,638425.0,655171.0,0.974440,84,7916889
1,03,339384.0,349336.0,0.971512,84,7916889
2,07,325157.0,334591.0,0.971804,84,7916889
3,15,145969.0,151615.0,0.962761,84,7916889
4,26,508006.0,522276.0,0.972677,84,7916889


In [71]:
df_final_alt['pop_ratio_mun_region'] = df_final_alt['sum_pop_municipal'] / df_final_alt['region_pop_municipal']
df_final_alt.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,region_pop_municipal,pop_ratio_mun_region
0,01,638425.0,655171.0,0.974440,84,7916889,0.080641
1,03,339384.0,349336.0,0.971512,84,7916889,0.042868
2,07,325157.0,334591.0,0.971804,84,7916889,0.041071
3,15,145969.0,151615.0,0.962761,84,7916889,0.018438
4,26,508006.0,522276.0,0.972677,84,7916889,0.064167


In [77]:
df_final_with_surf = df_final_alt.merge(df_surf_departments,
                                        left_on='code_department',
                                        right_on='code_insee')
df_final_with_surf.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,region_pop_municipal,pop_ratio_mun_region,code_insee,nom,surf_km2
0,01,638425.0,655171.0,0.974440,84,7916889,0.080641,01,Ain,5784.0
1,03,339384.0,349336.0,0.971512,84,7916889,0.042868,03,Allier,7379.0
2,07,325157.0,334591.0,0.971804,84,7916889,0.041071,07,Ardèche,5566.0
3,15,145969.0,151615.0,0.962761,84,7916889,0.018438,15,Cantal,5774.0
4,26,508006.0,522276.0,0.972677,84,7916889,0.064167,26,Drôme,6558.0


In [78]:
df_final_with_surf['densite'] = df_final_with_surf['sum_pop_municipal'] / df_final_with_surf['surf_km2']
df_final_with_surf.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,region_pop_municipal,pop_ratio_mun_region,code_insee,nom,surf_km2,densite
0,01,638425.0,655171.0,0.974440,84,7916889,0.080641,01,Ain,5784.0,110.377766
1,03,339384.0,349336.0,0.971512,84,7916889,0.042868,03,Allier,7379.0,45.993224
2,07,325157.0,334591.0,0.971804,84,7916889,0.041071,07,Ardèche,5566.0,58.418433
3,15,145969.0,151615.0,0.962761,84,7916889,0.018438,15,Cantal,5774.0,25.280395
4,26,508006.0,522276.0,0.972677,84,7916889,0.064167,26,Drôme,6558.0,77.463556


In [140]:
df_final_end = df_final_with_surf.drop(['region_pop_municipal', 'code_insee', 'surf_km2'],
                                       axis=1)
df_final_end['sum_pop_municipal'] = df_final_end['sum_pop_municipal'].astype(np.int64)
df_final_end['sum_pop_total'] = df_final_end['sum_pop_total'].astype(np.int64)
df_final_end['pop_ratio_mun_total'] = df_final_end['pop_ratio_mun_total'] * 100
df_final_end['pop_ratio_mun_region'] = df_final_end['pop_ratio_mun_region'] * 100
print(df_final_end.shape)
df_final_end.head()

(100, 8)


,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,pop_ratio_mun_region,nom,densite
0,01,638425,655171,97.444026,84,8.064089,Ain,110.377766
1,03,339384,349336,97.151167,84,4.286835,Allier,45.993224
2,07,325157,334591,97.180438,84,4.107131,Ardèche,58.418433
3,15,145969,151615,96.276094,84,1.843767,Cantal,25.280395
4,26,508006,522276,97.267728,84,6.416738,Drôme,77.463556


**Check results**

In [141]:
df_answers = pd.read_csv('https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson6/result-exo-cc.csv', sep=',')

In [142]:
df_answers.shape

(100, 8)

In [143]:
df_answers.head()

,nom_dept,code_dept,code_region,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
0,Ain,01,84,638425,655171,97.444026,8.064089,110.377766
1,Aisne,02,32,536136,549587,97.552526,8.925380,72.343274
2,Allier,03,84,339384,349336,97.151167,4.286835,45.993224
3,Alpes-de-Haute-Provence,04,93,162565,167331,97.151753,3.237103,23.246818
4,Hautes-Alpes,05,93,141107,146148,96.550757,2.809817,24.768650


In [144]:
df_final_end.head()

,code_department,sum_pop_municipal,sum_pop_total,pop_ratio_mun_total,code_region,pop_ratio_mun_region,nom,densite
0,01,638425,655171,97.444026,84,8.064089,Ain,110.377766
1,03,339384,349336,97.151167,84,4.286835,Allier,45.993224
2,07,325157,334591,97.180438,84,4.107131,Ardèche,58.418433
3,15,145969,151615,96.276094,84,1.843767,Cantal,25.280395
4,26,508006,522276,97.267728,84,6.416738,Drôme,77.463556


In [145]:
df_final_end_comp = \
    df_final_end.rename(dict(code_department='code_dept',
                             sum_pop_municipal='pop_municipale',
                             sum_pop_total='pop_totale',
                             pop_ratio_mun_total='pct_pop_municipale',
                             pop_ratio_mun_region='part_region',
                             nom='nom_dept'), axis=1) \
                .sort_values('code_dept').reset_index(drop=True)

**Both tables are the same!**

In [157]:
pd.testing.assert_frame_equal(df_final_end_comp.sort_index(axis=1), df_answers.sort_index(axis=1), check_dtype=False) == None

True

In [153]:
sorted(df_final_end_comp.columns.tolist())

['code_dept',
 'code_region',
 'densite',
 'nom_dept',
 'part_region',
 'pct_pop_municipale',
 'pop_municipale',
 'pop_totale']

In [154]:
sorted(df_answers.columns.tolist())

['code_dept',
 'code_region',
 'densite',
 'nom_dept',
 'part_region',
 'pct_pop_municipale',
 'pop_municipale',
 'pop_totale']